# 1. Analyze the data using the same techniques as for the last assignment.
Decide for yourself which and how to use the specific commands. Answer
the following questions in the report and include figures supporting your
answers:

In [ ]:
import pandas as pd

df = pd.read_csv('diamonds.csv')

In [ ]:
df.describe()

In [ ]:
df.corr()

## 1.1. Which classes exist? Are they (roughly) balanced?

## 1.2. Which noteworthy trends of features and relations between features as well as features and Classes do you see?

## 1.3. If you would need to distinguish the classes with those features, which features would you choose, any why?

# 2. In order to classify the three different Iris plant species, set up your first ML toolchain including the following steps:
• Data and Feature Preprocessing (if necessary and applicable)

## 2.1. Are there any outliers in the data which might need to be removed?

## 2.2. Are there any missing values which need to be taken care of?

## 2.3. Do you need to apply any feature preprocessing steps? (e.g Normalization, Feature Deletion/Reduction/Addition)

## 2.4. Are there any categorical features that need to be transformed so that it can be used for classification task?

## 2.5. Do you think it makes sense to derive any more features from the given ones? Why/why not?

# 3. Split up the dataset into a training and a separate held back test set in a clever way

## 3.1. Why is such a train/test split important?

## 3.2. Which train/test split percentage do you choose and why?

## 3.3. Think about how can you make sure to include samples from all three classes in both datasets and why this is important.

# 4. Train different classification models to distinguish between the three
Iris Plant Species:
– Use the following models: k Nearest Neighbour, Decision Tree,
Support Vector Machine

# 5. Use different hyperparameter settings for each model and explain why and how you chose them

# 6. Use an appropriate cross-validation setup for the training

# 7. Estimate the models’ performances on the held back test set:

# 8. Compare the models with their hyperparameter settings with two different error/performance measures * Why did you chose the specific error/performance measures? * What do they tell you?

## 8.1. Which model performs best with which hyperparameter settings and why do you think it does that way?

# 9. Explain which model you would use in deployment and why